In [1]:
import numpy as np
import pandas as pd

In [2]:
def find_entropy(df):
    Class=df.keys()[-1]
    values=df[Class].unique()
    entropy=0
    for value in values:
        prob=len(df[df[Class]==value])/len(df[Class])
        entropy+=-prob*np.log2(prob)
    return entropy

In [3]:
df=pd.read_csv('C:/Machine Learning/ML_Datasets/weather.csv')

In [4]:
find_entropy(df)

0.9402859586706309

In [5]:
def find_information_entropy(df,attribute):
    Class=df.keys()[-1]
    labels=df[Class].unique()
    attr_values=df[attribute].unique()
    avg_entropy=0
    for value1 in attr_values:
        entropy_subsample=0
        for value2 in labels:
            num=len(df[attribute][df[attribute]==value1][df[Class]==value2])
            den=len(df[attribute][df[attribute]==value1])
            prob=num/den
            entropy_subsample+=-(prob*np.log2(prob+1e-7))
        weight=den/len(df)
        avg_entropy+=weight*entropy_subsample
    return avg_entropy

In [6]:
find_information_entropy(df,'Humidity')

0.7884501687693255

In [7]:
def find_winner(df):
    IG=[]
    for attribute in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_information_entropy(df,attribute))
    return df.keys()[:-1][np.argmax(IG)]

In [8]:
find_winner(df)

'Outlook'

In [9]:
def training(df,tree=None):
    Class=df.keys()[-1]
    split_attr=find_winner(df)
    split_attr_values=df[split_attr].unique()
    if tree is None:
        tree={}
        tree[split_attr]={}
    for value in split_attr_values:
        sub_df=df[df[split_attr]==value].reset_index(drop=True)
        #Clvalue,count=np.unique(sub_df[Class],return_counts=True)
        Clvalue=sub_df[Class].unique()
        if len(Clvalue)==1:
            tree[split_attr][value]=Clvalue[0]
        else:
            tree[split_attr][value]=training(sub_df)
    return tree

In [10]:
tree=training(df)

In [11]:
import pprint
pprint.pprint(tree)

{'Outlook': {'overcast': 1,
             'rainy': {'Humidity': {'high': 0, 'normal': 1}},
             'sunny': {'Windy': {0: 1, 1: 0}}}}


In [12]:
def predict(inst,tree):
    for nodes in tree.keys():
        value=inst[nodes]
        tree=tree[nodes][value]
        prediction=0
        if type(tree) is dict:
            prediction=predict(inst,tree)
        else:
            prediction=tree
            break;
    return prediction

In [13]:
df1=pd.read_csv('C:/Machine Learning/ML_Datasets/weather_test.csv')
Y_label=[]
for i in range(len(df1)):
    inst=df1.iloc[i,:]
    prediction=predict(inst,tree)
    Y_label.append(prediction)
    

In [14]:
Y_label

[0, 1]

In [58]:
from sklearn import metrics
print(metrics.classification_report(df1.iloc[:,-1],Y_label))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

